In [1]:
#!/usr/bin/env python
# coding: utf-8

In [2]:


import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn

import scipy as sp
from scipy.linalg import svdvals

import numpy as np
#import powerlaw

import sklearn
from sklearn.decomposition import TruncatedSVD

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from torch import nn, Tensor

from torch.nn.utils import prune as prune

from torch.utils.data.sampler import SubsetRandomSampler

#import kneed as kneed


In [3]:
import numpy as np

def test_model(model):
    model.eval()
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    timings = []
    #GPU-WARM-UP
    i=0
    for data in test_loader:
        if(i>1000):
            break
        images, labels = data
        images = images.to(device)
        _ = model(images)
        i += 1
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            starter.record()
            outputs = model(images)
            ender.record()
            
            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings.append(curr_time)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: '+str(100 * correct / total))
    
    tot = np.sum(timings)
    mean_syn_per_batch = np.sum(timings) / len(timings)
    std_syn_per_batch = np.std(timings)
    print("Total inference time for test data: "+str(tot))
    print("Mean inference time per test batch: "+str(mean_syn_per_batch))
    print("Standard deviation of inference times per test batch: "+str(std_syn_per_batch))
    model.train()

In [4]:


device = "cpu"
if torch.cuda.is_available():
    device = "cuda"



model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
model = model.to(device)

torch.save(model, "./unpruned_model.pth")


/opt/conda/lib/python3.10/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/vgg/cifar10_vgg11_bn-eaeebf42.pt" to /root/.cache/torch/hub/checkpoints/cifar10_vgg11_bn-eaeebf42.pt
100%|██████████| 37.3M/37.3M [00:00<00:00, 136MB/s]


In [5]:


print(model.features)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace=True)
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU(inplace=True)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

In [6]:

train_transform = transforms.Compose([
                                 transforms.Pad(4),
                                 transforms.RandomCrop(32),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                             ])

test_transform = transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                             ])

#######################



train_set = torchvision.datasets.CIFAR10('./datasets', train=True, 
                                         download=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR10('./datasets', train=False, 
                                        download=True, transform=test_transform)
# Number of subprocesses to use for data loading
num_workers = 1
# How many samples per batch to load
batch_size = 1
# Percentage of training set to use as validation
valid_size = 0.5

num_test = len(test_set)
indices = list(range(num_test))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_test))
test_idx, valid_idx = indices[split:], indices[:split]

# Define samplers for obtaining training and validation batches
test_sampler = SubsetRandomSampler(test_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


# Prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                           num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(test_set, batch_size= 1, sampler=valid_sampler, 
                                           num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=2048, sampler=test_sampler, num_workers=num_workers)

100%|██████████| 170498071/170498071 [00:01<00:00, 102515277.38it/s]


Extracting ./datasets/cifar-10-python.tar.gz to ./datasets
Files already downloaded and verified


#### Testing the unpruned model:

In [7]:
test_model(model)

Accuracy of the network on the test images: 92.34
Total inference time for test data: 306.04524993896484
Mean inference time per test batch: 102.01508331298828
Standard deviation of inference times per test batch: 35.22032878017075


In [8]:


import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(valid_loader)


In [9]:


def ten2mat(tensor):
    r,c,ch,f = tensor.shape
    new_dim = [c,r*ch*f]
    return np.reshape(tensor, new_dim)

def eff_rank(matrix):
    frob = np.linalg.norm(matrix, 'fro')
    svals = svdvals(matrix)
    S = max(svals)
    r = frob/S
    return (r)


In [10]:


#utility functions


#reshape weight/feature tensor into a matrix
def ten2mat(tensor):
    r,c,ch,f = tensor.shape
    new_dim = [c,r*ch*f]
    return np.reshape(tensor, new_dim)
################################################################################################################
#Compute stable rank
def eff_rank(matrix):
    frob = np.linalg.norm(matrix, 'fro')
    svals = svdvals(matrix)
    S = max(svals)
    r = frob/S
    return (r)

################################################################################################################

# a dict to store the activations
activation = {}
def getActivation(name):
  # the hook signature
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

################################################################################################################

#Function to get conv+bn feature means
def get_mean_features(nc, num_classes, num_samples, conv_dict, model, dataiter):
    
    activation = {}
    def getActivation(name):
      # the hook signature
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    
    mat_features_list_conv = [ [] ]*(nc*num_classes)
    mat_features_list_bn = [ [] ]*(nc*num_classes)

    count = np.zeros((nc*num_classes))

    #with torch.no_grad():
    for i in range(num_samples):
        
        try:
            image,label = next(dataiter)
        except StopIteration:
            dataiter = iter(train_loader)
            image,label = next(dataiter)
        
        
        l2 =label.detach().numpy()
        l2 = l2[0]
        #print(l2)
        for j in range(nc):#range(len(conv_dict)):
            activation = {}
            z1 = conv_dict[j]
            z2 = z1 + 1

            #print(z1,z2)
            h1 = model.features[z1].register_forward_hook(getActivation('conv2d'))
            h2 = model.features[z2].register_forward_hook(getActivation('bn'))

            output = model(image.to(device))

            Xx1 = activation['conv2d']
            Xx2 = activation['bn']

            Xx1 = Xx1.to(device = 'cpu')
            Xx2 = Xx2.to(device = 'cpu')

            Xx01 = Xx1.detach().numpy()
            Xx02 = Xx2.detach().numpy()

            temp_mat1 = ten2mat(Xx01)
            temp_mat2 = ten2mat(Xx02)

            temp_idx = int(num_classes*j + l2)

            count[temp_idx]+=1
            #print(i, np.linalg.norm(temp_mat1),np.linalg.norm(temp_mat2), np.linalg.norm(temp_mat3), l2, j, z1, temp_idx)

            l31 = len(mat_features_list_conv[temp_idx])
            l32 = len(mat_features_list_bn[temp_idx])

            if l31 == 0:
                mat_features_list_conv[temp_idx] = temp_mat1

            else:
                mat_features_list_conv[temp_idx] = mat_features_list_conv[temp_idx] + temp_mat1

            if l32 == 0:
                mat_features_list_bn[temp_idx] = temp_mat2

            else:
                mat_features_list_bn[temp_idx] = mat_features_list_bn[temp_idx] + temp_mat2


            h1.remove()
            h2.remove()
       
    for i in range(nc*num_classes):
        mat_features_list_conv[i] = mat_features_list_conv[i] / count[i]
        mat_features_list_bn[i] = mat_features_list_bn[i] / count[i]
    
    return mat_features_list_conv, mat_features_list_bn, count


#################################################################################################################

#function to get differences between features

def get_features_diff(nc, num_classes, mat_features_list, conv_dict):
    
    mat_diff_list = []
    
    ll = int(0.5 * num_classes * (num_classes - 1))


    count_list = [ [[]]*ll ]*nc

    for k in range(nc):
        idx1 = 0
        idx2 = num_classes*k
        temp_list = [[]]*ll

        for i in range(num_classes):
            for j in range(i+1,num_classes):
                #print(idx1)
                ti = i+idx2
                tj = j+idx2
                temp1 = mat_features_list[ti] - mat_features_list[tj]
                
                temp_list[idx1] = temp1

                count_list[k][idx1]= (i,j)
                idx1+=1
        mat_diff_list.append([temp_list])

        
    return mat_diff_list

#################################################################################################################
        
def get_min_list(nc, num_classes, mat_diff_list):
    min_list = []
    ll = int(0.5 * num_classes * (num_classes - 1))

    for k in range(nc):
        layer_list = mat_diff_list[k][0]


        tempmat = layer_list[0]
        
        Y_conv = np.shape(tempmat)
        num_channels = Y_conv[0]
        #print(ll)
        temp_min_list= [[]]*num_channels


        for i in range(ll):
            temp1  = layer_list[i]



            temp1n = np.linalg.norm(temp1, axis=1)


            for j in range(num_channels):           
                if bool(temp_min_list[j]):
                    if temp_min_list[j] > temp1n[j]:
                        temp_min_list[j] = temp1n[j]                    
                else:
                    temp_min_list[j] = temp1n[j]




        min_list.append(temp_min_list)
    return min_list


################################################################################################################

def test(model, data_loader, device):
    acc = 0  # TODO compute top1
    correct_samples = 0
    total_samples = 0
    model.eval()
    with torch.no_grad():
        for (idx, (x, t)) in enumerate(data_loader):
            x = model.forward(x.to(device))
            t = t.to(device)
            _, indices = torch.max(x, 1)
            correct_samples += torch.sum(indices == t)
            total_samples += t.shape[0]

    acc = float(correct_samples) / total_samples
    return acc

################################################################################################################

def get_crit_vals(nc, min_list, crit_frac):
    minlist = []
    min_means = []
    min_crit = []
    for i in range(nc):
        temp_list = min_list[i] / np.max(min_list[i])
        mean = np.mean(temp_list)


        #print(i,mean_conv, mean_bn)

        crit = crit_frac* mean
        minlist.append(np.min(temp_list))
        min_means.append(mean)
        min_crit.append(crit)
    
    return min_list, min_means, min_crit

################################################################################################################

def get_pruning_mask2(nc, min_crit, min_list,model, conv_dict):
    
    Mask = [[]]*nc
    filters_pruned = [[]]*nc
    filters_remain = [[]]*nc
    
    for k in range(nc):


                #Conv output
        Scores= min_list[k] 
        Scores = Scores / np.max(Scores)
        len_sc = int(len(Scores))


        Criterion = min_crit[k]
        mask = np.ones(len_sc)
        for i in range(len(Scores)):
            temp = Scores[i]-Criterion
            if temp <= 0:
                mask[i]= 0

                
        num_rem = int(np.sum(mask))
        num_prn = int(len(mask) - num_rem)  
        
        filters_pruned[k] = num_prn
        filters_remain[k] = num_rem

        
        Test_ten = model.features[conv_dict[k]].weight
        
        Test_ten = Test_ten.to(device = 'cpu')

        Test_ten = Test_ten.detach().numpy()

        r,c,ch,f = np.shape(Test_ten)
        Test_Mask = [[]]*r
        One_ten = np.ones((c,ch,f))
        for i in range(r):
            Test_Mask[i] = One_ten*mask[i]
        X = torch.tensor(Test_Mask).to(device)
       
        
       
        
        Mask[k] = X
    return Mask, filters_pruned, filters_remain


#################################################################################################################

def prune_with_mask(nc, model, X, conv_dict):
    
    for k in range(nc):
        mm = prune.custom_from_mask(model.features[conv_dict[k]], name='weight', mask=X)
    
    return model

#################################################################################################################
def get_pruned_acc(nc, min_crit, min_list,model, conv_dict,test_loader,device):
    
    filters_pruned = [[]]*nc
    filters_remain = [[]]*nc
           
    for k in range(nc):
        #print(idx)
        
        
        #Conv output
        Scores = min_list[k] 
        Scores = Scores / np.max(Scores)
        len_sc = int(len(Scores))


        Criterion = min_crit[k]
        mask = np.ones(len_sc)
        for i in range(len(Scores)):
            temp = Scores[i]-Criterion
            if temp <= 0:
                mask[i]= 0

                
        num_rem = int(np.sum(mask))
        num_prn = int(len(mask) - num_rem)  
        
        filters_pruned[k] = num_prn
        filters_remain[k] = num_rem

      
        
        
        
        Test_ten = model.features[conv_dict[k]].weight
        
        Test_ten = Test_ten.to(device = 'cpu')

        Test_ten = Test_ten.detach().numpy()

        r,c,ch,f = np.shape(Test_ten)
        Test_Mask = [[]]*r
        One_ten = np.ones((c,ch,f))
        for i in range(r):
            Test_Mask[i] = One_ten*mask[i]
        X = torch.tensor(Test_Mask).to(device)
       
        m1 = prune.custom_from_mask( model.features[conv_dict[k]], name='weight', mask=X)
        
        
        #print(model_accs1[k, idx])
        #print(k, crit_frac, num_pruned, pf, T1)
    
    #device = torch.device('cuda')
    
    T1 = test(model, test_loader,device)
    T1 = 100*T1
    
    return model, T1


In [11]:
from urllib.request import HTTPError

U = 60

# accs = np.zeros(U)

accs = []

# num_pruned = np.zeros(U)
# num_remain = np.zeros(U)

num_pruned = []
num_remain = []


# conv_dict = [0,3,7,10,14,17,20,24,27,30, 34, 37, 40]
conv_dict = [0,4,8,11,15,18,22,25]
# conv_dict = [0,3,7,10,14,17,20,23,27,30,33,36,40,43,46,49]

nc = int(len(conv_dict))
num_classes = 10
num_samples = 1024

fracs = [.99,.9,.8, .7, .65,.6,.5,.4,.3,.2,.1,0]
Crit_frac = .5

threshold = 92.00
frac_count = 0
accuracy = 100000
uu=0

stop_at = 400
freq = {}

while Crit_frac > 0:
    #print(uu)
    mat_features_list_conv, mat_features_list_bn, count = get_mean_features(nc, num_classes, num_samples, conv_dict, model, dataiter) 
    diff_list_conv = get_features_diff(nc, num_classes, mat_features_list_conv, conv_dict)
    diff_list_bn = get_features_diff(nc, num_classes, mat_features_list_bn, conv_dict)
    min_list_conv = get_min_list(nc, num_classes, diff_list_conv)
    min_list_bn = get_min_list(nc, num_classes, diff_list_bn)
   
    minlist_bn, minmeans_bn, min_crit_bn = get_crit_vals(nc, min_list_bn, Crit_frac)
    
    flag = True
    test_model1 = ""
    while(flag):
        try:
            test_model1 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
            flag = False
            #test_model1 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg16_bn", pretrained=True)
            #test_model1 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg11_bn", pretrained=True)
        except HTTPError:
            print("Error while downloading pretrained model... retrying...")
            flag = True


    test_model1 = test_model1.to(device)
    
    Mask_bn, filters_pruned_bn, filters_remain_bn = get_pruning_mask2(nc, min_crit_bn, min_list_bn,test_model1, conv_dict)
    
#     num_pruned[uu] = np.sum(filters_pruned_bn)
#     num_remain[uu] = np.sum(filters_remain_bn)
    fpbn = np.sum(filters_pruned_bn)
    frbn = np.sum(filters_remain_bn)
    np.append(num_pruned, fpbn)
    np.append(num_remain, frbn)
    
    pf = 100 * (fpbn / (fpbn + frbn))
    
    for k in range(nc):
        mm = prune.custom_from_mask( test_model1.features[conv_dict[k]], name='weight', mask=Mask_bn[k])
        mm = prune.remove(mm, name='weight')
    
    T2 = test(test_model1, test_loader, device)
    accuracy = T2 * 100
    #accs[uu] = T2 *100
    np.append(accs,accuracy)
    
    if accuracy > threshold:
        for k in range(nc):
            mmm = prune.custom_from_mask( model.features[conv_dict[k]], name='weight', mask=Mask_bn[k])
            mmm = prune.remove(mmm, name='weight')
            
        
        print(Crit_frac, uu, T2*100, pf, 100-pf)
    else:
        print(Crit_frac, uu, T2*100, pf, 100-pf)
        frac_count +=1
        Crit_frac = np.maximum(0, Crit_frac - 0.025)
        
    uu +=1
    
    if Crit_frac in freq:
        freq[Crit_frac] += 1
        if freq[Crit_frac] == stop_at:
            break
    else:
        freq[Crit_frac] = 0
        
    
    
    
    


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
/tmp/ipykernel_24/4180631273.py:267: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  X = torch.tensor(Test_Mask).to(device)


0.5 0 91.9 10.065406976744185 89.93459302325581


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


0.475 1 92.30000000000001 8.502906976744185 91.49709302325581


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


...

0.024999999999999842 280 92.02 12.42732558139535 87.57267441860465


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


0.024999999999999842 281 92.02 12.42732558139535 87.57267441860465


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


0.024999999999999842 282 91.97999999999999 12.463662790697674 87.53633720930233


In [12]:
torch.save(model, "./pruned_model.pth")

#### Testing the pruned model (with only zeroed out weights):

In [13]:
test_model(model)

Accuracy of the network on the test images: 81.7
Total inference time for test data: 343.84238052368164
Mean inference time per test batch: 114.61412684122722
Standard deviation of inference times per test batch: 41.75067729054353


In [14]:


a_file = open("test_vgg11_cifar10.txt", "a")


np.savetxt(a_file, [filters_pruned_bn], fmt = '%1.1i')

np.savetxt(a_file, [filters_remain_bn], fmt = '%1.1i')

np.savetxt(a_file, [accuracy], fmt = '%1.4f')


a_file.close()



#### Performing architecture modifications:

In [15]:
!pip install torch-pruning
import torch_pruning as tp
    
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d): #Iterating over all the conv2d layers of the model
        channel_indices = [] #Stores indices of the channels to prune within this conv layer
        t = module.weight.clone().detach()
        t = t.reshape(t.shape[0], -1)
        z = torch.all(t == 0, dim=1)
        z = z.tolist()
        
        for i, flag in enumerate(z):
            if(flag):
                channel_indices.append(i)

        if(channel_indices == []):
            continue
        
        # 1. build dependency graph for vgg
        DG = tp.DependencyGraph().build_dependency(model, example_inputs=torch.randn(1,3,32,32).to(device))

        # 2. Specify the to-be-pruned channels. Here we prune those channels indexed by idxs.
        group = DG.get_pruning_group(module, tp.prune_conv_out_channels, idxs=channel_indices)
        #print(group)

        # 3. prune all grouped layers that are coupled with the conv layer (included).
        if DG.check_pruning_group(group): # avoid full pruning, i.e., channels=0.
            group.prune()
    
# 4. Save & Load
model.zero_grad() # We don't want to store gradient information
torch.save(model, './arch_pruned_model.pth') # without .state_dict

#### Testing the pruned model (after architecture modifications)

In [16]:
test_model(model)

Accuracy of the network on the test images: 81.58
Total inference time for test data: 315.049373626709
Mean inference time per test batch: 105.01645787556966
Standard deviation of inference times per test batch: 34.3397031397702
